In [3]:
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import AlignIO
import pandas as pd

# Make unique genbank files from multi-gbk

In [3]:
ids=[]
for record in SeqIO.parse('Cmag_ref_GLOBAL_multi-GenBank.gbff','genbank'):
    print(record.id)
    ids+=[record.id]

AP014555.1
10_12_13_a
4-3_5-3_6-
C._Magnifi
11_MAR-806
1_2?3_3?3_
4_5_14_kil
1_23_33_di
9-3_17_BR-
A._phaseol
A._mariana
8_pac_mt_I
9_pac_mt_I
15_16_giga


In [4]:
dic={'AP014555.1':'P.okutanii_ref',
 '10_12_13_a':'A.phaseoliformis',
 '4-3_5-3_6-':'P.extenta',
 'C._Magnifi':'C.magnifica',
 '11_MAR-806':'A.southwardae',
 '1_2?3_3?3_':'A.diagonalis', # doublon to be eliminated
 '4_5_14_kil':'P.soyae',
 '1_23_33_di':'A.diagonalis',
 '9-3_17_BR-':'X.pliocardia',
 'A._phaseol':'A.phaseoliformis_ref', # reference AP014557.1
 'A._mariana':'A.mariana_ref',
 '8_pac_mt_I':'C.pacifica',
 '9_pac_mt_I':'C.rectimargo',
 '15_16_giga':'A.gigas'}

cols=['A.phaseoliformis_ref', 'P.soyae','C.magnifica','C.pacifica','A.phaseoliformis', 'X.pliocardia','C.rectimargo', 'A.southwardae','A.mariana_ref','A.diagonalis','P.extenta','A.gigas','P.okutanii_ref']


In [71]:
seq1=record.seq[feature.location.start:]
def find_end_of_gene(seq1,gene_start):
    for i in range(gene_start, len(seq1), 3):
        if seq1[i:i+3] in ['TAG','TAA']:
            return(i+3)
            break
find_end_of_gene(seq1)

TypeError: find_end_of_gene() missing 1 required positional argument: 'gene_start'

In [82]:
recs={}
seqs={}
# seqs={'ND2': {}, 'ND6': {}, 'ND4L': {}, 'COX3': {}, 'COX1': {}, 'COX2': {}, 'CYTB': {}, 'ATP8': {}, 'ND4': {}, 'ATP6': {}, 'ND3': {}, 'ND1': {}, 'ND5': {}}
def find_end_of_gene(seq1,gene_start):
    for i in range(gene_start, len(seq1), 3):
        if seq1[i:i+3] in ['TAG','TAA']:
            return(i+3)
            break

for record in SeqIO.parse('Cmag_ref_GLOBAL_multi-GenBank_manually_annotated.gbff','genbank'):
    if record.id=='1_2?3_3?3_':
        continue
        
    i=0
    recs[dic[record.id]]={}
    record.name=dic[record.id]
    for feature in record.features:
        if feature.type=='source':
            continue
        if feature.type=='gene':
            i+=1
            feature.qualifiers['locus_tag']=record.name[0]+record.name[2:5]+'_'+"{:02d}".format(i)
            last_locus_tag=record.name[0]+record.name[2:5]+'_'+"{:02d}".format(i)
        else:
            feature.qualifiers['locus_tag']=last_locus_tag
        if feature.type=='CDS':
            if feature.qualifiers['gene'][0]=='ND2':
                recs[record.name].update({feature.qualifiers['gene'][0]:feature.qualifiers['locus_tag']})
#                 print(feature,len(feature.extract(record.seq))%3)
#                 print(len(feature.extract(record.seq)))
                print(feature)
                print(feature.location.start,feature.location.end)
                print(feature.location.end-feature.location.start)
 
                prot_seq=feature.extract(record.seq).translate(table=5, cds=True)

                gene_start=feature.location.start
                gene_end=find_end_of_gene(record.seq,gene_start)
                print('gene_end',gene_end)
                nucl_seq=record.seq[gene_start:gene_end]
                print(len(nucl_seq))
                prot_seq=nucl_seq.translate(table=5, cds=True)
                print (prot_seq)
#             prot_seq=feature.extract(record.seq).translate(table=5, cds=True)

#             seqs[feature.qualifiers['locus_tag']]={"prot":prot_seq,"nucl":nucl_seq}

#     break
#     SeqIO.write(record, dic[record.id]+'.gbk', "genbank")
     

type: CDS
location: [1553:2585](+)
qualifiers:
    Key: gene, Value: ['ND2']
    Key: locus_tag, Value: Poku_06

1553 2585
1032
gene_end 2585
1032
MLIGWFVPMATIIGGYIMVGGIFVCVYSVGLLGMWVGMELSFFGAISVLGGSSVEEVEGVQKYFIVQVFSSIFLLIGFLLVVNFYSGGWLVEFILVVGLMVKLGLFPFYFWIPSVMGVVSWVGCFVVSVAQKFNPIWILVNLMLEGYLQQVIEFSACLTGLVGCIGGIGVLQFRVLLAFSSMVHLGYMVFMSLIGWDVFLSYMGLYFCLSFSLMMSLWVCGVYSFLDLMKSNSGGEMNLFMISLYMFSFAGVPPLSGCFMKAVFLMSCWELFPIGCVILFLSSGVSLFYYSSFFMYLNLYWGSVLKWEFVSGKFGDLKVFLFFVSVLLNVAMGFSLFLLIVLV
type: CDS
location: [13750:14782](+)
qualifiers:
    Key: gene, Value: ['ND2']
    Key: locus_tag, Value: Apha_44
    Key: translation, Value: ['MLIG*FVPMAVIVRGYIMVGGIFVCIYRVGLLGIWVGIELRFFGAVGVLGGSRVEEVEGVQKYFIIQVFRSIFLLIGFLLVINFYKGG*LVEFIIVTGLIMKLGLFPFYF*IPSVIRVVSWVRCFVVSVVQKFNPI*ILVNLILESYLQQIIEFSACLTGLVGCIGGIGVLQFRVLLAFSSMVHLGYIIFICLVGWDEFLTYIILYFCLSFRLIMSLWVCGIYRFLDLIKSNGGRDVDLFMISLYIFSFAGIPPLSGCFMKAVFLVSCWEVFPVGCVVLFLSSGVSLFYYSRFFTYLNLY*GRILKWQFVKRRLADLKVFLFFVRVLLNVVLGFGLFLLIVII']

13750 14782
1032


TranslationError: Final codon 'GTA' is not a stop codon

In [84]:

find_end_of_gene(record.seq,feature.location.start)

16058

In [ ]:
df=pd.DataFrame.from_dict(recs,orient='index')
df.columns



# Add locus_tags to geneious files

In [11]:
dic={'AP014555.1':'P.okutanii_ref',
 '10_12_13_a':'A.phaseoliformis',
 '4-3_5-3_6-':'P.extenta',
 'C._Magnifi':'C.magnifica',
 '11_MAR-806':'A.southwardae',
 '1_2?3_3?3_':'A.diagonalis', # doublon to be eliminated
 '4_5_14_kil':'P.soyae',
 '1_23_33_di':'A.diagonalis',
 '9-3_17_BR-':'X.pliocardia',
 'A._phaseol':'A.phaseoliformis_ref', # reference AP014557.1
 'A._mariana':'A.mariana_ref',
 '8_pac_mt_I':'C.pacifica',
 '9_pac_mt_I':'C.rectimargo',
 '15_16_giga':'A.gigas'}

dic={'Pac_II_':'C.rectimargo',
 '11_MAR':'A.southwardae',
 'extenta_combined':'P.extenta',
 'A._phaseoliformi':'A.phaseoliformis_ref',
 'R_magnifica':'C.magnifica',
 '8-3_Blake_Ridge':'X.pliocardia',
 'A._mariana':'A.mariana_ref',
 'Contig_54_11_MAR':'A.southwardae_c54',
 '13_Aleutian':'A.phaseoliformis',
 'diagonalis':'A.diagonalis',
 'Pac_I_':'C.pacifica',
 'P._okutanii':'P.okutanii_ref',
 'kilmeri':'P.soyae'}

In [479]:
from os import listdir
from os.path import isfile, join
mypath='./geneious_mitoch/'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-3:]=='.gb']
recs={}
print(onlyfiles)
for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        i=0
        record.id=dic[record.name]
        recs[record.id]={}
        for n in range(len(record.features)):
            
            feature=record.features[n]
            if n==0:
                if feature.type=='source':
                    feature.qualifiers['organism']=dic[record.name]
                    continue
                else:
                    print(len(record.seq))
                    new_feature=SeqFeature(FeatureLocation(1, len(record.seq)), type="source")
                    new_feature.qualifiers['organism']= dic[record.name]
                    continue
            if feature.type=='misc_feature':
                continue
            if feature.type=='tRNA':
                continue
            if feature.type=='CDS':
                i+=1
                try:
                    feature.qualifiers['product']=feature.qualifiers['standard_name']
                except KeyError:
                    pass
                feature.qualifiers['locus_tag']=record.id[0]+record.id[2:5]+'_'+"{:02d}".format(i)
                last_locus_tag=record.id[0]+record.id[2:5]+'_'+"{:02d}".format(i)
#                 print(feature.qualifiers.keys())
                recs[record.id].update({feature.qualifiers['product'][0].split(' ')[0]:feature.qualifiers['locus_tag']})
            else:
#                 print(feature.qualifiers.keys())
                try:
                    feature.qualifiers['product']=feature.qualifiers['standard_name']
                except KeyError:
                    pass
#                 feature.qualifiers['locus_tag']=last_locus_tag
        if new_feature:
#             print(new_feature)
            record.features=[new_feature]+record.features
#         SeqIO.write(record, path+'locus_taged_'+record.name+'.gbk', "genbank")
#     break
recs    

TypeError: 'str' object is not callable

In [69]:

df=pd.DataFrame.from_dict(recs,orient='index')
df

,ND6,ND4L,ND2,COX3,COX1,COX,CYTB,ATP8,ND4,ATP6,ND3,ND1,ND5,COX2,cytochrome,ATP,NADH
C.rectimargo,Crec_01,Crec_02,Crec_03,Crec_04,Crec_05,Crec_06,Crec_07,Crec_08,Crec_09,Crec_10,Crec_11,Crec_12,Crec_13,NaN,NaN,NaN,NaN
A.southwardae,Asou_13,Asou_01,Asou_02,Asou_03,Asou_04,Asou_05,Asou_06,Asou_07,Asou_08,Asou_09,Asou_10,Asou_11,Asou_12,NaN,NaN,NaN,NaN
P.extenta,Pext_01,Pext_02,Pext_03,Pext_04,Pext_05,NaN,Pext_07,Pext_08,Pext_09,Pext_10,Pext_11,Pext_12,Pext_13,Pext_06,NaN,NaN,NaN
A.phaseoliformis_ref,Apha_10,Apha_11,Apha_12,Apha_13,Apha_01,NaN,Apha_03,Apha_04,Apha_05,Apha_06,Apha_07,Apha_08,Apha_09,Apha_02,NaN,NaN,NaN
X.pliocardia,Xpli_01,Xpli_02,Xpli_03,Xpli_04,Xpli_05,NaN,Xpli_07,Xpli_08,Xpli_09,Xpli_10,Xpli_11,Xpli_12,Xpli_13,Xpli_06,NaN,NaN,NaN
A.mariana_ref,Amar_01,Amar_02,Amar_03,Amar_04,Amar_05,NaN,Amar_07,Amar_08,Amar_09,Amar_10,Amar_11,Amar_12,Amar_13,Amar_06,NaN,NaN,NaN
A.southwardae_c54,Asou_12,NaN,Asou_01,Asou_02,Asou_03,NaN,Asou_05,Asou_06,Asou_07,Asou_08,Asou_09,Asou_10,Asou_11,Asou_04,NaN,NaN,NaN
A.phaseoliformis,Apha_13,Apha_12,Apha_11,Apha_10,Apha_09,NaN,Apha_07,Apha_06,Apha_05,Apha_04,Apha_03,Apha_02,Apha_01,Apha_08,NaN,NaN,NaN
A.diagonalis,Adia_01,Adia_02,Adia_03,Adia_04,Adia_05,NaN,Adia_07,Adia_08,Adia_09,Adia_10,Adia_11,Adia_12,Adia_13,Adia_06,NaN,NaN,NaN
P.okutanii_ref,Poku_01,Poku_02,Poku_03,Poku_04,Poku_05,NaN,Poku_07,Poku_08,Poku_09,Poku_10,Poku_11,Poku_12,Poku_13,Poku_06,NaN,NaN,NaN


# Fix genes on geneious mitoch

In [107]:
from os import listdir
from os.path import isfile, join
mypath='./geneious_mitoch/'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-4:]=='.gbk']
print(onlyfiles)
recs={}

for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        for feature in record.features:
            if feature.type in ['CDS']:
                print(file)
                print(feature.qualifiers['product'])
                print(feature.location.start,feature.location.end)
                nucl_seq=feature.extract(record.seq)
                print(nucl_seq)
                prot_seq=feature.extract(record.seq).translate(table=5, cds=True)



['locus_taged_extenta_combined.gbk', 'locus_taged_A._phaseoliformi.gbk', 'locus_taged_P._okutanii.gbk', 'locus_taged_Pac_II_.gbk', 'locus_taged_diagonalis.gbk', 'locus_taged_11_MAR.gbk', 'locus_taged_R_magnifica.gbk', 'locus_taged_13_Aleutian.gbk', 'locus_taged_Contig_54_11_MAR.gbk', 'locus_taged_kilmeri.gbk', 'locus_taged_8-3_Blake_Ridge.gbk', 'locus_taged_A._mariana.gbk', 'locus_taged_Pac_I_.gbk']
locus_taged_extenta_combined.gbk
['ND6 CDS']
2057 2537
ATGGCAGAAGTATTTGTTTTGGTATTTTTGCTTTTATGTAGGAATTTTTCAATTTGGGTAGAGCACCCTCTTATTTTGGGGAGAGGGCTATTGGGGGTGAGAATTGTGAGATTGCCCCTAATAATTTCTGTTGGTAGACTTTTCGGGTTCAGGTTTTTTATTGTGGTGGTTGGAGGAGTTTTAATTGTTTTTGCTTACTCAGTATCTTTAATTTCTGTAACTAAGAGCGATCTGTTATTGAAATATTTTACTCGGGGAGTGGGTATAGTTGGGAAATTTTGAACGGTTTTTTTATATTTTAGGTTGGTTTTTTTATTATCGAGGTGGGTGGTGGGAAGGGGGGTATTTTGTTGAAGGGATGTCTTATATTTTAGGAAGGGGTGGGGCTTTGGGGTAGTTTTGCTTGGCTTTTTATTGTTAGCTATTATGGTGATGGCTATTGGGTTGGCGAGGAAATTCAAAGGGGCCTTAATTAAATAA
locus_taged_extenta_combined.gbk
['ND4L CDS']
2555 2825
GTGTT

TranslationError: Sequence length 851 is not a multiple of three

# CLEAN mitochondrial genomes from geneious

## Make fasta from geneious gbk and fix record name

In [112]:
dic={'Pac_II_':'C.rectimargo',
 '11_MAR':'A.southwardae',
 'extenta_combined':'P.extenta',
 'A._phaseoliformi':'A.phaseoliformis_ref',
 'R_magnifica':'C.magnifica',
 '8-3_Blake_Ridge':'X.pliocardia',
 'A._mariana':'A.mariana_ref',
 'Contig_54_11_MAR':'A.southwardae_c54',
 '13_Aleutian':'A.phaseoliformis',
 'diagonalis':'A.diagonalis',
 'Pac_I_':'C.pacifica',
 'P._okutanii':'P.okutanii_ref',
 'kilmeri':'P.soyae'}

In [113]:
from os import listdir
from os.path import isfile, join
mypath='./geneious_mitoch/original_annotations/'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-3:]=='.gb']
for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        record.id=dic[record.name]
        SeqIO.write(record,'./geneious_mitoch/'+record.id+'.fasta','fasta')

/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/GenBank/__init__.py:366: BiopythonParserWarning: Attempting to fix invalid location '18468..45' as it looks like incorrect origin wrapping. Please fix input file, this could have unintended behavior.
  BiopythonParserWarning,
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/GenBank/__init__.py:366: BiopythonParserWarning: Attempting to fix invalid location '18507..45' as it looks like incorrect origin wrapping. Please fix input file, this could have unintended behavior.
  BiopythonParserWarning,
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/GenBank/__init__.py:366: BiopythonParserWarning: Attempting to fix invalid location '17868..193' as it looks like incorrect origin wrapping. Please fix input file, this could have unintended behavior.
  BiopythonParserWarning,
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/GenBank/__init__.py:366: BiopythonParserWarning: 

## Redo the anotations using GeSeq - Annotation of Organellar Genomes and Rmag as reference

## Parse multigbk file

In [2]:
dic={'ref_Pokuta':'P.okutanii_ref',
 'Aphaseolif':'A.phaseoliformis',
 'Pextenta-4':'P.extenta',
 'ref_Cmagni':'C.magnifica',
 'Asouthward':'A.southwardae',
 'Psoyae-7%2':'P.soyae',
 'Adiagonali':'A.diagonalis',
 'Xpliocardi':'X.pliocardia',
 'ref_Aphase':'A.phaseoliformis_ref', # reference AP014557.1
 'ref_Amaria':'A.mariana_ref',
 'Cpacifica-':'C.pacifica',
 'Crectimarg':'C.rectimargo',
 '15_16_giga':'A.gigas'}

dic2={'P.okutanii_ref': 'AP014555.1',
 'A.phaseoliformis': '10_12_13_a',
 'P.extenta': '4-3_5-3_6-',
 'C.magnifica': 'C._Magnifi',
 'A.southwardae': '11_MAR-806',
 'A.diagonalis': '1_23_33_di',
 'P.soyae': '4_5_14_kil',
 'X.pliocardia': '9-3_17_BR-',
 'A.phaseoliformis_ref': 'AP014557.1',
 'A.mariana_ref': 'A._mariana',
 'C.pacifica': '8_pac_mt_I',
 'C.rectimargo': '9_pac_mt_I',
 'A.gigas': '15_16_giga'}

In [170]:
ids=[]
for record in SeqIO.parse('geneious_mitoch/Geneious_mitoch_GLOBAL_multi-GenBank.gbff','genbank'):
    # fix header
    record.name= dic2[dic[record.id]]
    record.description=dic[record.id]+' mitochondrion'
    print(record.description)
    i=0
    for feature in record.features:
        if feature.type=='source':
            continue
        if feature.type=='gene':
            i+=1
            feature.qualifiers['locus_tag']=record.description[0]+record.description[2:5]+'_'+"{:02d}".format(i)
            last_locus_tag=record.description[0]+record.description[2:5]+'_'+"{:02d}".format(i)
        else:
            feature.qualifiers['locus_tag']=last_locus_tag
        if feature.type=='CDS':
            try:
                feature.qualifiers['translation']=feature.extract(record.seq).translate(table=5, cds=True)
            except:
                pass
    SeqIO.write(record, dic[record.id]+'.gbk', "genbank")

A.southwardae mitochondrion
P.okutanii_ref mitochondrion
C.rectimargo mitochondrion
P.soyae mitochondrion
C.pacifica mitochondrion
A.phaseoliformis mitochondrion
C.magnifica mitochondrion
P.extenta mitochondrion
A.phaseoliformis_ref mitochondrion
X.pliocardia mitochondrion
A.mariana_ref mitochondrion
A.diagonalis mitochondrion


## read gbk files and fix CDS
This was done manually so the script is garbage

In [152]:
from os import listdir
from os.path import isfile, join

mypath='./geneious_mitoch/'
mypath='./'

path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-4:]=='.gbk']
print(onlyfiles)

#########################
def find_end_of_gene(seq,gene_start):
    for i in range(gene_start, len(seq), 3):
        if seq[i:i+3] in ['TAG','TAA']:
            return(i+3)
            break
#####################           

for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        features=[]
        print (record.description)
        for feature in record.features:
            if feature.type=='CDS':
                if feature.qualifiers['locus_tag'][0] == 'Rpha_51':
                    print('>')
                    print(feature.extract(record.seq))
                
                #-------
                try:
                    feature.qualifiers['translation']=feature.extract(record.seq).translate(table=5, cds=True)
                
                except:
#                     if feature.qualifiers['locus_tag'][0] in ['Apha_38','Cpac_16','Psoy_16','Asou_14','Crec_14']:
#                         continue
                    print(feature.qualifiers['gene'],feature.qualifiers['locus_tag'])
                    nucl_seq=feature.extract(record.seq)
                    gene_start=feature.location.start
                    gene_end=feature.location.end
                    print(gene_start,gene_end)
                    print(record.seq[gene_start-3:gene_start+3],record.seq[gene_end-3:gene_end+3])
                    print(find_end_of_gene(record.seq,gene_start))
                    print(nucl_seq)
                    prot_seq=feature.extract(record.seq).translate(table=5, cds=True)
                    print(prot_seq)
                    
    SeqIO.write(record, dic[record.id]+'.gbk', "genbank")    

['A.gigas_shifted.gbk']
15_16_giga mitochondrion


In [138]:
# record = SeqIO.read(mypath+file,'genbank')
# record=record[-30:]+record[:-30]
# SeqIO.write(record, mypath+file[:-4]+'_shifted.gbk', "genbank") 

### 

In [463]:
### Get the reverse complement of Phaseoliphormis
mypath='./'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-4:]=='.gbk']
print(onlyfiles)

#####################           

for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        record=record.reverse_complement()
        
        SeqIO.write(record, file[:-4]+'_rc.gbk', "genbank")    

['C.magnifica.gbk', 'P.okutanii_ref.gbk', 'A.diagonalis.gbk', 'A.phaseoliformis_ref.gbk', 'P.extenta.gbk', 'A.phaseoliformis.gbk', 'C.pacifica.gbk', 'A.phaseoliformis_rc.gbk']


# get codon alignments

In [79]:
recs={}
seqs={}
# seqs={'ND2': {}, 'ND6': {}, 'ND4L': {}, 'COX3': {}, 'COX1': {}, 'COX2': {}, 'CYTB': {}, 'ATP8': {}, 'ND4': {}, 'ATP6': {}, 'ND3': {}, 'ND1': {}, 'ND5': {}}
mypath='./geneious_mitoch/'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-4:]=='.gbk']
print(onlyfiles)

for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
#         print(dic[record.id])
        recs[dic[record.id]]={}
        for feature in record.features:
            if feature.type=='CDS':

                recs[dic[record.id]].update({feature.qualifiers['gene'][0]:feature.qualifiers['locus_tag'][0]})

df=pd.DataFrame.from_dict(recs,orient='index')
df


['A.gigas.gbk', 'C.magnifica.gbk', 'P.okutanii_ref.gbk', 'A.diagonalis.gbk', 'A.phaseoliformis_ref.gbk', 'P.extenta.gbk', 'A.phaseoliformis.gbk', 'C.pacifica.gbk', 'P.soyae.gbk', 'A.mariana_ref.gbk', 'A.southwardae.gbk', 'X.pliocardia.gbk', 'C.rectimargo.gbk']


In [201]:
record= SeqIO.read(mypath+'Z.Svelum.gb', "genbank")
from Bio.Alphabet import Gapped,IUPAC
from Bio.Alphabet.IUPAC import ExtendedIUPACProtein

Svel={}
for feature in record.features:
    if feature.type=='CDS':
        gene=feature.qualifiers['gene'][0]
        header='Z.Svelum|'+feature.qualifiers['protein_id'][0]
        print (feature.qualifiers)
        vel_aa=SeqRecord(Seq(feature.qualifiers['translation'][0],Gapped(ExtendedIUPACProtein(), '-')),id=header,name=header)
        vel_nucl=SeqRecord(feature.location.extract(record).seq,id=header,name=header)
        Svel[gene]={'faa':vel_aa,'fna':vel_nucl}

                   
Svel    


OrderedDict([('gene', ['COX1']), ('codon_start', ['1']), ('transl_table', ['5']), ('product', ['cytochrome c oxidase subunit 1']), ('protein_id', ['AFG18166.1']), ('translation', ['MRWLFSTNHKDIGTLYILFAIWAGMVGTSLSLLIRAELGQPGALLGDDQLYNVIVTAHAFIMIFFLVMPMMMGGFGNWFVPMMLGAPDMAFPRMNNMSFWLLPPALTLLLGSAAVESGAGTGWTVYPPLSGNLAHAGASVDLTIFSLHLAGASSIMASINFITTAINMRSRGMRFERMPLFVWSIKITAVLLLLSLPVLAGAITMLLTGRNFNTSFFDPAGGGDPILYQHLFWFFGHPEVYILILPGFGMISHIIGHHAGKKEVFGTLGMIYAMMAIGILGFIVWAHHMFTVGMDVDTRAYFTAATMIIAVPTGIKVFSWLATIYGSRIKYETPMVWALGFIFLFTIGGLTGVVLANSSLDIMMHDTYYVVAHFHYVLSMGAVFALFAAFNYWFPLMTGLTLHPIWTKAHFFIMFIGVNLTFFPQHFLGLSGMPRRYSDYPDCFTKWNVLSSFGSMISFISVLYFMFIVWEAFSAQRAVIWSSYVGTDTEWDDLLPVDFHNLSETTALVRA'])])
OrderedDict([('gene', ['COX2']), ('codon_start', ['1']), ('transl_table', ['5']), ('product', ['cytochrome c oxidase subunit 2']), ('protein_id', ['AFG18167.1']), ('translation', ['MLFWGQLGLQDAVSPLMEQLVFFHDHAMLILVMVASLIGYAIVSLIMSSFTCRYILEGQQIETVWTIAPAIVLVFLALPSLRLLYLLDEVNDPVLTIKTVGHQWYWSYEYSDFNSLEIDSY

{'COX1': {'faa': SeqRecord(seq=Seq('MRWLFSTNHKDIGTLYILFAIWAGMVGTSLSLLIRAELGQPGALLGDDQLYNVI...VRA', Gapped(ExtendedIUPACProtein(), '-')), id='Z.Svelum|AFG18166.1', name='Z.Svelum|AFG18166.1', description='<unknown description>', dbxrefs=[]),
  'fna': SeqRecord(seq=Seq('ATGCGTTGGTTATTTTCTACCAATCACAAAGATATTGGAACTTTATATATTTTA...TAA', IUPACAmbiguousDNA()), id='Z.Svelum|AFG18166.1', name='Z.Svelum|AFG18166.1', description='<unknown description>', dbxrefs=[])},
 'COX2': {'faa': SeqRecord(seq=Seq('MLFWGQLGLQDAVSPLMEQLVFFHDHAMLILVMVASLIGYAIVSLIMSSFTCRY...HAA', Gapped(ExtendedIUPACProtein(), '-')), id='Z.Svelum|AFG18167.1', name='Z.Svelum|AFG18167.1', description='<unknown description>', dbxrefs=[]),
  'fna': SeqRecord(seq=Seq('ATGCTCTTTTGAGGACAATTAGGACTACAAGATGCTGTATCTCCATTAATAGAA...TAA', IUPACAmbiguousDNA()), id='Z.Svelum|AFG18167.1', name='Z.Svelum|AFG18167.1', description='<unknown description>', dbxrefs=[])},
 'ND4L': {'faa': SeqRecord(seq=Seq('MTIFPIMFLCGTICFWSAMITLMFQYKHLLSMLLSLEAMMLSLFIM

In [208]:
from Bio import Alphabet
from Bio.Alphabet import IUPAC,Gapped
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import MuscleCommandline
from Bio.codonalign import build
from Bio.Data.CodonTable import generic_by_id,unambiguous_dna_by_id
import subprocess
import sys

i=0
table5=unambiguous_dna_by_id[5]
table5.nucleotide_alphabet=IUPAC.ambiguous_dna
table5.protein_alphabet=IUPAC.extended_protein
table5.forward_table['AAN'] = "K"
table5.forward_table['GAY'] = "D"
table5.forward_table['YTA'] = "L"
table5.forward_table['ATY'] = "I"
table5.forward_table['TCY'] = "S"
table5.forward_table['GTY'] = "V"
table5.forward_table['GGY'] = "G"
table5.forward_table['CCY'] = "P"
table5.forward_table['GGR'] = "G"
for gene in df.columns.values:
    i+=1
    j=0
    print(gene)
    fna=[]
    faa=[]
    for index in df.index.values:
        j+=1
        locus_tag=df[gene][index]
        record = SeqIO.read(mypath+index+'.gbk', "genbank")
        for feature in record.features:
            if feature.type != 'CDS':
                continue
            if feature.qualifiers['locus_tag'][0]==locus_tag:
                
                ### create SeqRecord object ####
                DNAseq=feature.location.extract(record).seq
                AAseq=Seq(feature.qualifiers['translation'][0],alphabet=Gapped(IUPAC.extended_protein))
                header='|'.join([index,locus_tag])

                fna_seq = SeqRecord(DNAseq,id=header)
                faa_seq = SeqRecord(AAseq,id=header)

                fna.append(fna_seq)
                faa.append(faa_seq)
                if j==1:
                    fna.append(Svel[gene]['fna'])
                    faa.append(Svel[gene]['faa'])

    faa=[f for f in sorted(faa, key=lambda x : x.id)]
    fna=[f for f in sorted(fna, key=lambda x : x.id)]
#     for rec in fna:
#         print(rec)
    #### run MUSCLE ####
    muscle_cline = MuscleCommandline(clwstrict=True)
    child = subprocess.Popen(str(muscle_cline),
                     stdin=subprocess.PIPE,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE,
                     universal_newlines=True,
                     shell=(sys.platform!="win32"))
    SeqIO.write(faa, child.stdin, "fasta")
    child.stdin.close()
    align = AlignIO.read(child.stdout, "clustal",alphabet=Gapped(IUPAC.extended_protein))
    aln = MultipleSeqAlignment([f for f in sorted(align, key=lambda x : x.id)])
    codon_aln = build(aln, fna,complete_protein=False,max_score=30,codon_table=table5)
    print(codon_aln)
    print(len(codon_aln))

#     SeqIO.write(codon_aln, './geneious_mitoch/Codons_aln/'+gene+'.fna', "fasta")
    SeqIO.write(faa, './geneious_mitoch/Codons_aln/'+gene+'.faa', "fasta")

    n=0
    names={}
    for seq in codon_aln:
        seq.id=seq.id.split('|')[0]
        seq.name=seq.id
        names[n]=seq.name.replace('.','')
        n+=1
    if i==1:
        cat=codon_aln
    if i>1:
        cat.sort()
        codon_aln.sort()
        cat+=codon_aln
for i in range(len(cat)):
    cat[i].id=names[i]
print(cat)

SeqIO.write(cat, './geneious_mitoch/Codons_aln/cat_mitoch_genes.fna', "fasta")
    
    

COX1
CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 14 rows and 1803 columns (601 codons)
ATATTTGTGGTTGAGCGTGTAGCTAAAAGTCTTATATATGTTTATAAATTTGTATTT...GTT A.diagonalis|Adia_22
---------------------------------ATATCTGTTTATAAATTTGTATTT...GTT A.gigas|Agig_01
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.mariana_ref|Amar_19
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.phaseoliformis_ref|Apha_55
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.phaseoliformis|Apha_35
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.southwardae|Asou_18
ATATTTGTGGTTGAGCGTGTAGCTAAGGGTCTTATGTCTGTTTATAAGTTTGTGTTT...GTT C.magnifica|Cmag_01
ATGTTTGTGGTTGAGCGTGTGATTAAAAGTTTTATACTTATTTACAAGTTTGTATTT...GTT C.pacifica|Cpac_22
ATATTTGTGCCTGAGCGTGTGGCTAAAATTTTTATATTTGTTTATAAGTTTATGTTT...GTT C.rectimargo|Crec_21
ATATTTGTGGTTGAGCGTGTAGCTAAAAGTCTTATATCTGTTTATAAATTTGTATTT...GTT P.extenta|Pext_21
ATATTTGTGGTTGAGCGTGTAGCTAAAAGTCTTATATCTGTT

CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 14 rows and 945 columns (315 codons)
------------GTGATTAGGGGGATAATTGTGGTTGTTGTTATAATAATATCTGTA...TTA A.diagonalis|Adia_45
------------GTGATCAGGGGGTTAATTGTAGTTATTGTTATAATAATATCTGTA...CTA A.gigas|Agig_24
------------GTGATTAGGGGGATAATTGTAGTTATTATTATAATAATGGCTGTA...TTA A.mariana_ref|Amar_43
------------GTGATTAGGGGGATAATTGTAGTTATTATTATAATAATGTCTGTA...TTA A.phaseoliformis_ref|Apha_24
------------GTGATTAGGGGGATAATTGTAGTTATTATTATAATAATGTCTGTA...TTA A.phaseoliformis|Apha_11
------------GTGGTTAGGGGAATAATTGTAGTTATTATTATAATAATGTCTGTA...TTA A.southwardae|Asou_43
------------ATGATTAGAGGGATAATTGTAGTCATTGTTATAATAGTGTCTGTA...CTA C.magnifica|Cmag_22
------------GTGATTAGGGGTATAATTGTAGTTATTGTTATAATAGTGTCTGTA...TTG C.pacifica|Cpac_42
------------GTGATTAGGGGTATAATTGTAGTTATTGTTATAATAGTGTCTGTA...TTG C.rectimargo|Crec_47
------------GTGATTAGGGGGATAATTGTAGTTGTTATTATAATAATATCTGTA...TTA P.extenta|Pext_44
------------GTGGTTAGGGGGATAATTGTAGTTATTATTATAATA

14

In [83]:
from Bio import Alphabet
from Bio.Alphabet import IUPAC,Gapped
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import MuscleCommandline
from Bio.codonalign import build
from Bio.Data.CodonTable import generic_by_id,unambiguous_dna_by_id,register_ncbi_table
import subprocess
import sys
import json

pd.options.display.max_rows = 4000

table5=generic_by_id[5]
table5.nucleotide_alphabet=IUPAC.ambiguous_dna
table5.protein_alphabet=IUPAC.extended_protein
table5.forward_table['AAN'] = "N"
codon_aln = codonalign.build(aln, sequence, codon_table=default_codon_table)

# print(table5.forward_table)
register_ncbi_table(table5)

Table 5 Invertebrate Mitochondrial, SGC4

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA W   | A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L   | CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA M(s)| ACA T   | AAA K   | AGA S   | A
A | ATG M(s)| ACG T   | AAG K   | AGG S   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V(s)| GCG A   | GAG E   | GGG G 

TypeError: register_ncbi_table() missing 5 required positional arguments: 'alt_name', 'id', 'table', 'start_codons', and 'stop_codons'

# Fix gbk file from GeSeq (Mitobim)

In [103]:
from os import listdir
from os.path import isfile, join
mypath='./mitobim_mitoch/'
path=mypath
onlyfiles = [f for f in listdir(mypath) if f[-3:]=='.gb']
print(onlyfiles)
sampleid=['_'.join(file.split('_')[2:4]) for file in onlyfiles]

dic=dict(zip(['A.phaseoliformis', 'P.extenta',  'A.gigas2','C.rectimargo','A.southwardae', 'P.soyae1','A.gigas1', 'A.diagonalis', 'X.pliocardia', 'C.pacifica',  'P.soyae2'],['Aphaseolif_bb','PeNtenta-4_bb', 'Consensus_Vgig2Mitobim', 'Crectimarg_bb', 'Asouthward_bb', 'Psoyae1-7%252_bb', 'Consensus_Vgig1Mitobim', 'Adiagonali_bb', 'Xpliocardi_bb', 'Cpacifica-_bb', 'Psoyae2-7%252_bb']))

dic2=dict([v,k] for k,v in dic.items())
print(dic2)
print(dic)
dic2['PeNtenta-4_bb']

['mitobim_assemblies_PeNtenta-4_bb_bb_bb_GenBank.gb', 'mitobim_assemblies_Aphaseolif_bb_bb_bb_GenBank.gb', 'mitobim_all_Consensus_Vgig2Mitobim_Node2Spades_GenBank.gb', 'mitobim_assemblies_Crectimarg_bb_bb_bb_bb_GenBank.gb', 'mitobim_assemblies_Psoyae1-7%252_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb', 'mitobim_all_Consensus_Vgig1Mitobim_Node2Spades_GenBank.gb', 'mitobim_assemblies_Adiagonali_bb_bb_bb_bb_bb_bb_bb_GenBank.gb', 'mitobim_assemblies_Xpliocardi_bb_bb_bb_bb_bb_GenBank.gb', 'Rsou_shifted_Asouthward_bb_bb_bb_GenBank.gb', 'mitobim_assemblies_Cpacifica-_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb', 'mitobim_assemblies_Psoyae2-7%252_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb']
{'Aphaseolif_bb': 'A.phaseoliformis', 'PeNtenta-4_bb': 'P.extenta', 'Consensus_Vgig2Mitobim': 'A.gigas2', 'Crectimarg_bb': 'C.rectimargo', 'Asouthward_bb': 'A.southwardae', 'Psoyae1-7%252_bb': 'P.soyae1', 'Consensus_Vgig1Mitobim': 'A.gigas1', 'Adiagonali_bb': 'A.diagonalis', 'Xpliocardi_bb': 'X.pliocardia', 'Cpa

'P.extenta'

In [146]:
def find_end_of_gene(seq,gene_start):
    for i in range(gene_start, len(seq), 3):
        if seq[i:i+3] in ['TAG','TAA']:
            return(i+3)
            break
def find_end_of_partial_cds(seq,gene_start):
    for i in range(gene_start, len(seq), 3):
        if seq[i:i+3] in ['NNN']:
            return(i+3)
            break
            
for file in onlyfiles:
    
    for record in SeqIO.parse(path+file,'genbank'):
        # fix header
        record.id='_'.join(file.split('_')[2:4])
#         print(record.id)
        record.name= dic2[record.id]
        print(record.name)
        record.description=record.name+' mitochondrion'
        print(file)
        i=0
        for feature in record.features:
            if feature.type=='source':
                continue
            if feature.type=='gene':
                i+=1
                feature.qualifiers['locus_tag']=record.description[0]+record.description[2:5]+'_'+"{:02d}".format(i)
                last_locus_tag=record.description[0]+record.description[2:5]+'_'+"{:02d}".format(i)
#                 print(last_locus_tag)
            else:
                feature.qualifiers['locus_tag']=last_locus_tag
            if feature.type=='CDS':
                try:
                    feature.qualifiers['translation']=feature.extract(record.seq).translate(table=5, cds=True)
                except:
                    print('!!! issue with cds')
#                     print(feature)
                    nucl_seq=feature.extract(record.seq)
                    gene_start=feature.location.start
                    gene_end=feature.location.end

                    if feature.qualifiers['locus_tag']in ['Agig_30','Agig_33','Agig_55']:

                        gene_end=find_end_of_partial_cds(record.seq,gene_start)
                        feature.location=FeatureLocation(gene_start, gene_end, feature.location.strand)
                        nucl_seq=record.seq[gene_start:gene_end-4]
                        nucl_seq=nucl_seq[:-(len(nucl_seq)%3)]
                        prot_seq=nucl_seq.translate(table=5, cds=False)
                        feature.qualifiers['translation']=prot_seq
                        feature.qualifiers['note']=['Partial coding sequence resolved due to low depth']
#                         print(feature)



#                     print(find_end_of_gene(record.seq,gene_start))
#                     print(nucl_seq)
#                     prot_seq=record.seq[gene_start,gene_end-3].translate(table=5, cds=True)
#                     print(prot_seq)
                    
                    
                    
#         SeqIO.write(record, path+record.name+'_annotated'+'.gbk', "genbank")
        if record.name=='A.gigas2':
            record=record[:3111]+record[3118:]
            SeqIO.write(record, path+record.name+'_annotated'+'.gbk', "genbank")

            SeqIO.write(record, path+record.name+'.fasta', "fasta")

P.extenta
mitobim_assemblies_PeNtenta-4_bb_bb_bb_GenBank.gb
A.phaseoliformis
mitobim_assemblies_Aphaseolif_bb_bb_bb_GenBank.gb
A.gigas2
mitobim_all_Consensus_Vgig2Mitobim_Node2Spades_GenBank.gb
!!! issue with cds
C.rectimargo
mitobim_assemblies_Crectimarg_bb_bb_bb_bb_GenBank.gb
P.soyae1
mitobim_assemblies_Psoyae1-7%252_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb
A.gigas1
mitobim_all_Consensus_Vgig1Mitobim_Node2Spades_GenBank.gb
!!! issue with cds
!!! issue with cds
A.diagonalis
mitobim_assemblies_Adiagonali_bb_bb_bb_bb_bb_bb_bb_GenBank.gb
X.pliocardia
mitobim_assemblies_Xpliocardi_bb_bb_bb_bb_bb_GenBank.gb
A.southwardae
Rsou_shifted_Asouthward_bb_bb_bb_GenBank.gb
C.pacifica
mitobim_assemblies_Cpacifica-_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb
P.soyae2
mitobim_assemblies_Psoyae2-7%252_bb_bb_bb_bb_bb_bb_bb_bb_bb_GenBank.gb


In [136]:
mypath='./geneious_mitoch/'
path='./mitobim_mitoch/'
filedic={}
recs={}
seqs={}
# seqs={'ND2': {}, 'ND6': {}, 'ND4L': {}, 'COX3': {}, 'COX1': {}, 'COX2': {}, 'CYTB': {}, 'ATP8': {}, 'ND4': {}, 'ATP6': {}, 'ND3': {}, 'ND1': {}, 'ND5': {}}
allfiles = [path+f for f in listdir(path) if f[-4:]=='.gbk']+[mypath+'C.magnifica.gbk',
 mypath+'P.okutanii_ref.gbk', mypath+'A.phaseoliformis_ref.gbk',  mypath+'A.mariana_ref.gbk']
print(allfiles)
for file in allfiles:
    print(file)
    print(file.split('/')[2].replace('_annotated','').replace('.gbk',''))
    for record in SeqIO.parse(file,'genbank'):
        record.id=file.split('/')[2].replace('_annotated','').replace('.gbk','')
        filedic[record.id]=file
        recs[record.id]={}
        for feature in record.features:
            if feature.type=='CDS':

                recs[record.id].update({feature.qualifiers['gene'][0]:feature.qualifiers['locus_tag'][0]})

df=pd.DataFrame.from_dict(recs,orient='index')
df


['./mitobim_mitoch/A.gigas1_annotated.gbk', './mitobim_mitoch/A.gigas2_annotated.gbk', './mitobim_mitoch/P.extenta_annotated.gbk', './mitobim_mitoch/A.phaseoliformis_annotated.gbk', './mitobim_mitoch/P.soyae2_annotated.gbk', './mitobim_mitoch/C.pacifica_annotated.gbk', './mitobim_mitoch/P.soyae1_annotated.gbk', './mitobim_mitoch/X.pliocardia_annotated.gbk', './mitobim_mitoch/C.rectimargo_annotated.gbk', './mitobim_mitoch/A.southwardae_annotated.gbk', './mitobim_mitoch/A.diagonalis_annotated.gbk', './geneious_mitoch/C.magnifica.gbk', './geneious_mitoch/P.okutanii_ref.gbk', './geneious_mitoch/A.phaseoliformis_ref.gbk', './geneious_mitoch/A.mariana_ref.gbk']
./mitobim_mitoch/A.gigas1_annotated.gbk
A.gigas1
./mitobim_mitoch/A.gigas2_annotated.gbk
A.gigas2
./mitobim_mitoch/P.extenta_annotated.gbk
P.extenta
./mitobim_mitoch/A.phaseoliformis_annotated.gbk
A.phaseoliformis
./mitobim_mitoch/P.soyae2_annotated.gbk
P.soyae2
./mitobim_mitoch/C.pacifica_annotated.gbk
C.pacifica
./mitobim_mitoch/P.s

,COX1,COX2,CYTB,ATP8,ND4,ATP6,ND3,ND1,ND5,ND6,ND4L,ND2,COX3
A.gigas1,Agig_01,Agig_04,Agig_07,Agig_11,Agig_12,Agig_20,Agig_21,Agig_28,Agig_33,Agig_36,Agig_37,Agig_41,Agig_55
A.gigas2,Agig_03,Agig_06,Agig_09,Agig_13,Agig_14,Agig_22,Agig_23,Agig_30,Agig_35,Agig_38,Agig_39,Agig_43,Agig_57
P.extenta,Pext_24,Pext_27,Pext_31,Pext_34,Pext_35,Pext_42,Pext_43,Pext_51,Pext_56,Pext_02,Pext_03,Pext_07,Pext_20
A.phaseoliformis,Apha_24,Apha_27,Apha_29,Apha_35,Apha_36,Apha_43,Apha_44,Apha_53,Apha_58,Apha_04,Apha_05,Apha_09,Apha_20
P.soyae2,Psoy_23,Psoy_26,Psoy_31,Psoy_34,Psoy_35,Psoy_42,Psoy_43,Psoy_50,Psoy_55,Psoy_01,Psoy_02,Psoy_06,Psoy_18
C.pacifica,Cpac_27,Cpac_29,Cpac_32,Cpac_36,Cpac_37,Cpac_43,Cpac_44,Cpac_52,Cpac_57,Cpac_04,Cpac_05,Cpac_08,Cpac_20
P.soyae1,Psoy_23,Psoy_26,Psoy_31,Psoy_34,Psoy_35,Psoy_42,Psoy_43,Psoy_50,Psoy_55,Psoy_01,Psoy_02,Psoy_06,Psoy_18
X.pliocardia,Xpli_32,Xpli_34,Xpli_38,Xpli_42,Xpli_43,Xpli_50,Xpli_51,Xpli_58,Xpli_63,Xpli_10,Xpli_11,Xpli_14,Xpli_26
C.rectimargo,Crec_21,Crec_24,Crec_27,Crec_30,Crec_31,Crec_38,Crec_39,Crec_48,Crec_53,Crec_01,Crec_02,Crec_04,Crec_14
A.southwardae,Asou_23,Asou_26,Asou_28,Asou_33,Asou_34,Asou_40,Asou_41,Asou_51,Asou_56,Asou_01,Asou_02,Asou_06,Asou_18


In [137]:
filedic

{'A.gigas1': './mitobim_mitoch/A.gigas1_annotated.gbk',
 'A.gigas2': './mitobim_mitoch/A.gigas2_annotated.gbk',
 'P.extenta': './mitobim_mitoch/P.extenta_annotated.gbk',
 'A.phaseoliformis': './mitobim_mitoch/A.phaseoliformis_annotated.gbk',
 'P.soyae2': './mitobim_mitoch/P.soyae2_annotated.gbk',
 'C.pacifica': './mitobim_mitoch/C.pacifica_annotated.gbk',
 'P.soyae1': './mitobim_mitoch/P.soyae1_annotated.gbk',
 'X.pliocardia': './mitobim_mitoch/X.pliocardia_annotated.gbk',
 'C.rectimargo': './mitobim_mitoch/C.rectimargo_annotated.gbk',
 'A.southwardae': './mitobim_mitoch/A.southwardae_annotated.gbk',
 'A.diagonalis': './mitobim_mitoch/A.diagonalis_annotated.gbk',
 'C.magnifica': './geneious_mitoch/C.magnifica.gbk',
 'P.okutanii_ref': './geneious_mitoch/P.okutanii_ref.gbk',
 'A.phaseoliformis_ref': './geneious_mitoch/A.phaseoliformis_ref.gbk',
 'A.mariana_ref': './geneious_mitoch/A.mariana_ref.gbk'}

In [138]:
from Bio import Alphabet
from Bio.Alphabet import IUPAC,Gapped
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import MuscleCommandline
from Bio.codonalign import build
from Bio.Data.CodonTable import generic_by_id,unambiguous_dna_by_id
import subprocess
import sys

i=0
table5=unambiguous_dna_by_id[5]
table5.nucleotide_alphabet=IUPAC.ambiguous_dna
table5.protein_alphabet=IUPAC.extended_protein
table5.forward_table['AAN'] = "K"
table5.forward_table['GAY'] = "D"
table5.forward_table['YTA'] = "L"
table5.forward_table['ATY'] = "I"
table5.forward_table['TCY'] = "S"
table5.forward_table['GTY'] = "V"
table5.forward_table['GGY'] = "G"
table5.forward_table['CCY'] = "P"
table5.forward_table['GGR'] = "G"
for gene in df.columns.values:
    i+=1
    j=0
    print(gene)
    fna=[]
    faa=[]
    for index in df.index.values:
        j+=1
        locus_tag=df[gene][index]
        record = SeqIO.read(filedic[index], "genbank")
        for feature in record.features:
            if feature.type != 'CDS':
                continue
            if feature.qualifiers['locus_tag'][0]==locus_tag:
                
                ### create SeqRecord object ####
                DNAseq=feature.location.extract(record).seq
                AAseq=Seq(feature.qualifiers['translation'][0],alphabet=Gapped(IUPAC.extended_protein))
                header='|'.join([index,locus_tag])

                fna_seq = SeqRecord(DNAseq,id=header)
                faa_seq = SeqRecord(AAseq,id=header)

                fna.append(fna_seq)
                faa.append(faa_seq)
#                 if j==1:
#                     fna.append(Svel[gene]['fna'])
#                     faa.append(Svel[gene]['faa'])

    faa=[f for f in sorted(faa, key=lambda x : x.id)]
    fna=[f for f in sorted(fna, key=lambda x : x.id)]
#     for rec in fna:
#         print(rec)
    #### run MUSCLE ####
    muscle_cline = MuscleCommandline(clwstrict=True)
    child = subprocess.Popen(str(muscle_cline),
                     stdin=subprocess.PIPE,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE,
                     universal_newlines=True,
                     shell=(sys.platform!="win32"))
    SeqIO.write(faa, child.stdin, "fasta")
    child.stdin.close()
    align = AlignIO.read(child.stdout, "clustal",alphabet=Gapped(IUPAC.extended_protein))
    aln = MultipleSeqAlignment([f for f in sorted(align, key=lambda x : x.id)])
    codon_aln = build(aln, fna,complete_protein=False,max_score=30,codon_table=table5)
    print(codon_aln)
    print(len(codon_aln))

    SeqIO.write(codon_aln, './mitobim_mitoch/Codon_aln/'+gene+'.fna', "fasta")
    SeqIO.write(faa, './mitobim_mitoch/Codon_aln/'+gene+'.faa', "fasta")

    n=0
    names={}
    for seq in codon_aln:
        seq.id=seq.id.split('|')[0]
        seq.name=seq.id
        names[n]=seq.name.replace('.','')
        n+=1
    if i==1:
        cat=codon_aln
    if i>1:
        cat.sort()
        codon_aln.sort()
        cat+=codon_aln
for i in range(len(cat)):
    cat[i].id=names[i]
print(cat)

SeqIO.write(cat, './mitobim_mitoch/Codon_aln/cat_mitoch_genes.fna', "fasta")

COX1
CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 15 rows and 1803 columns (601 codons)
ATATTTGTGGTTGAGCGTGTAGCTAAAAGTCTTATATATGTTTATAAATTTGTATTT...GTT A.diagonalis|Adia_48
ATGTTTGTGGTTGAGCGTGTGGCTAAAAGTCTTATATCTGTTTATAAATTTGTATTT...GTT A.gigas1|Agig_01
ATGTTTGTGGTTGAGCGTGTGGCTAAAAGTCTTATATCTGTTTATAAATTTGTATTT...GTT A.gigas2|Agig_03
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.mariana_ref|Amar_19
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.phaseoliformis_ref|Apha_55
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.phaseoliformis|Apha_24
ATATTTGTGGTTGAGCGTGTAGCTAAGAGTCTTATAGCTGTTTATAAATTTGTGTTT...GTT A.southwardae|Asou_23
ATATTTGTGGTTGAGCGTGTAGCTAAGGGTCTTATGTCTGTTTATAAGTTTGTGTTT...GTT C.magnifica|Cmag_01
ATGTTTGTGGTTGAGCGTGTGATTAAAAGTTTTATACTTATTTACAAGTTTGTATTT...GTT C.pacifica|Cpac_27
ATATTTGTGCCTGAGCGTGTGGCTAAAATTTTTATATTTGTTTATAAGTTTATGTTT...GTT C.rectimargo|Crec_21
ATATTTGTGGTTGAGCGTGTAGCTAAAAGTCTTATATCTGTT

/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: C.pacifica|Cpac_29(M 0) does not correspond to C.pacifica|Cpac_29(TTG)
  BiopythonWarning)


CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 15 rows and 1485 columns (495 codons)
---------ATGGGATCTTGGAATAAGTTATGGTTTTTTGATATAAACACTGTTGTT...--- A.diagonalis|Adia_50
---------ATGGGATCTTGGAATAAGTTATGGTTTTTTGATATAAACACTGTTGTT...--- A.gigas1|Agig_04
---------ATGGGATCTTGGAATAAGTTATGGTTTTTTGATATAAACACTGTTGTT...--- A.gigas2|Agig_06
---------ATGGGGTCTTGAAATAAGTTATGATTTTATGATATAAATACTGTTGTT...--- A.mariana_ref|Amar_21
---------ATGGGGTCTTGAAATAAGTTATGATTTTATGATATAAATACTGTTGTT...--- A.phaseoliformis_ref|Apha_02
---------ATGGGGTCTTGAAATAAGTTATGATTTTATGATATAAATACTGTTGTT...--- A.phaseoliformis|Apha_27
---------ATGGGGTCTTGAAATAAGTTATGATTTTATGATATAAATACTGTTGTT...--- A.southwardae|Asou_26
---------ATGGGATCTTGAAACAAATTATGGTTTTTTGATATAAATACTGTGGTT...--- C.magnifica|Cmag_03
TTGTTTTGAATGGGATCTTGAGAGAAAATGTGATTTTATGATATAAATACTGTTGTC...GTC C.pacifica|Cpac_29
---------ATGGGATCCTGAGAGAAATTTTGGTTTTATGATATAAATACTGTTGTC...GAC C.rectimargo|Crec_24
---------ATGGGGTCTTGAAGTAAGTTATGGTTTTTTGATATAAA

/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: P.extenta|Pext_34(M 0) does not correspond to P.extenta|Pext_34(GTG)
  BiopythonWarning)
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: P.okutanii_ref|Poku_29(M 0) does not correspond to P.okutanii_ref|Poku_29(GTG)
  BiopythonWarning)
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: P.soyae1|Psoy_34(M 0) does not correspond to P.soyae1|Psoy_34(GTG)
  BiopythonWarning)
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: P.soyae2|Psoy_34(M 0) does not correspond to P.soyae2|Psoy_34(GTG)
  BiopythonWarning)


CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 15 rows and 114 columns (38 codons)
ATGCCTCAGTTTTCTCCTCTTTTTGTTGATTTTATTTTTTTGTTCTATTGGGCGGTG...--- A.diagonalis|Adia_57
ATGCCTCAGTTTTCTCCTCTTTTTGTTGATTTTATTTTTTTGTTCTATTGGGCGGTG...--- A.gigas1|Agig_11
ATGCCTCAGTTTTCTCCTCTTTTTGTTGATTTTATTTTTTTGTTCTATTGGGCGGTG...--- A.gigas2|Agig_13
ATGCCTCAATTTTCACCTCTTTTTGTCGATTTTATTTTTTTATTTTATTGGGCGGTG...--- A.mariana_ref|Amar_28
ATGCCTCAATTTTCACCTCTTTTTGTTGATTTTATTTTTTTATTTTATTGGGCGGTA...--- A.phaseoliformis_ref|Apha_09
ATGCCTCAATTTTCACCTCTTTTTGTTGATTTTATTTTTTTATTTTATTGGGCGGTA...--- A.phaseoliformis|Apha_35
ATGCCTCAATTTTCACCTCTTTTTGTTGATTTTATTTTTTTATTTTATTGGGCGGTG...--- A.southwardae|Asou_33
ATGCCTCAGTTTTCACCTCTTTTTGTAGATTTTATTTTTTTATTTTATTGGGCGGTG...--- C.magnifica|Cmag_07
ATGCCTCAGTTTTCACCTCTTTTTGTTGATTTTATTTTTTTGTTTTATTGGGCGGTG...--- C.pacifica|Cpac_36
ATGCCTCAGTTTTCACCTCTATTTGTGGATTTTATTTTTTTGTTTTATTGGGCGGTG...--- C.rectimargo|Crec_30
GTGCCTCAGTTTTCACCTCTTTTTGTTGATTTTATTTTTATGTTTTATT

/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: X.pliocardia|Xpli_50(M 0) does not correspond to X.pliocardia|Xpli_50(GTG)
  BiopythonWarning)


CodonAlphabet(Invertebrate Mitochondrial) CodonAlignment with 15 rows and 711 columns (237 codons)
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGGGGATTTAGGGTACTT...TTG A.diagonalis|Adia_65
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATGGGTGGATTTAGAGTTCTT...TTA A.gigas1|Agig_20
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATGGGTGGATTTAGAGTTCTT...TTA A.gigas2|Agig_22
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGTGGATTTAGAGTGTTG...TTG A.mariana_ref|Amar_35
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGTGGATTTAGAGTGTTG...TTG A.phaseoliformis_ref|Apha_16
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGTGGATTTAGAGTGTTG...TTG A.phaseoliformis|Apha_43
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGTGGATTTAGAGTGTTA...TTG A.southwardae|Asou_40
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGTGGCTTTAGGGTGCTA...TTG C.magnifica|Cmag_13
ATGGTTAGGGATATTTTTACTTTTATGGATTGTTTTATAGGGGGATTTAGGGTATTA...TTA C.pacifica|Cpac_43
ATGGTTAGGGATATTTTCACTTTTATGGATTGTTTTATAGGGGGATTTAGGGTGTTA...TTA C.rectimargo|Crec_38
ATGACTAGGGATATTTTTACTTTTATGGATTGTTTTATGGGCGGGTTT

/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: A.diagonalis|Adia_67(M 0) does not correspond to A.diagonalis|Adia_67(ATT)
  BiopythonWarning)
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: A.gigas1|Agig_21(M 0) does not correspond to A.gigas1|Agig_21(ATT)
  BiopythonWarning)
/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/codonalign/__init__.py:579: BiopythonWarning: A.gigas2|Agig_23(M 0) does not correspond to A.gigas2|Agig_23(ATT)
  BiopythonWarning)


ValueError: Sequence contain codon not in the alphabet (NNN)! 

In [117]:
record[gene_start:gene_end].seq

Seq('TATTATCTTTAACATTTGCTTCTATAGCTTATATAAAGTGAAGAGCTTTTAACT...GGA', IUPACAmbiguousDNA())

# Clean up Geneious gbk files and generate Codon-based aln

In [17]:
from os import listdir
from os.path import isfile, join
mypath='./mitobim_mitoch/batch_export/'
onlyfiles = [f for f in listdir(mypath) if f[-3:]=='.gb']
onlyfiles
# SeqIO.convert(mypath+onlyfiles[0],'genbank',mypath+'test.gbk','genbank')

['X.pliocardia.gb',
 'A.southwardae.gb',
 'P.extenta.gb',
 'A.gigas2.gb',
 'A.diagonalis.gb',
 'P.soyoae2.gb',
 'C.rectimargo.gb',
 'P.soyoae1.gb',
 'C.pacifica.gb',
 'A.gigas1.gb',
 'A.phaseoliformis.gb']

In [38]:
for file in [onlyfiles[3],onlyfiles[9]]:
    print(file)
    if file=='A.gigas1.gb':
        for record in SeqIO.parse(mypath+file,'genbank'):
            print(len(record))
            newrecord=record[11173:]+record[:11173]
            print(len(newrecord))
            print(newrecord[:10].seq)
            print(newrecord[-10:].seq)
    if file=='A.gigas2.gb':
        for record in SeqIO.parse(mypath+file,'genbank'):
            print(len(record))
            newrecord=record[11559:]+record[:11559]
            print(len(newrecord))
            print(newrecord[:10].seq)
            print(newrecord[-10:].seq)
#     SeqIO.write(newrecord,file[:-3]+'.gbk','genbank')
        

A.gigas2.gb
15759
15759
GTGTAGTTTT
AAGTTTTTTN
A.gigas1.gb
15575
15575
TTTTTACTTC
NNNNNNNNNN


In [39]:
for file in [onlyfiles[3],onlyfiles[9]]:
    print(file)
    for record in SeqIO.parse(mypath+file,'genbank'):
        print(len(record))
    

A.gigas2.gb
15759
A.gigas1.gb
14826


/Users/maeperez/anaconda2/envs/py36/lib/python3.6/site-packages/Bio/GenBank/__init__.py:1398: BiopythonParserWarning: Expected sequence length 15426, found 14826 (Consensus_Vgig1Mitobim).
  BiopythonParserWarning,


In [24]:
for file in onlyfiles:
    for record in SeqIO.parse(mypath+file,'genbank'):
        record.id=file[0]+file[1:-3]
        print(record.description)
        i=0
        for feature in record.features:
            if feature.type=='source':
                continue
            if feature.type!='gene':
                i+=1
                feature.qualifiers['locus_tag']=record.id[0]+record.id[2:5]+'_'+"{:02d}".format(i)
                last_locus_tag=record.id[0]+record.id[2:5]+'_'+"{:02d}".format(i)
#                 print(last_locus_tag)
            else:
                feature.qualifiers['locus_tag']=last_locus_tag
            if feature.type=='CDS':
                try:
                    feature.qualifiers['translation']=feature.extract(record.seq).translate(table=5, cds=True)
                except:
                    print('!!! issue with cds')
                    print(feature.qualifiers['locus_tag'])
                    print(feature.qualifiers)
                    feature.qualifiers['translation']=feature.extract(record.seq).translate(table=5, cds=True)

                    break
            

Pliocardia sp. isolate 17_S18 mitochondrial DNA, partial sequence
Abyssogena southwardae isolate 11_S2 mitochondrion, complete genome
Phreagena extenta isolate 18_S21 mitochondrial DNA, partial sequence
Archivesica gigas isolate 15_S12 mitochondrial DNA, partial sequence
!!! issue with cds
Agig_14
OrderedDict([('gene', ['ND3']), ('locus_tag', 'Agig_14'), ('note', ["Derived using Geneious Prime 2020.2.2 'Annotate from Database' based on nucleotide similarity"])])


TranslationError: Sequence length 301 is not a multiple of three